<a href="https://colab.research.google.com/github/CrankyTitanO7/envidia1/blob/total/envidia_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gpt-generated ai use and training

may need more computing power.

##usage script

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add the pad token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Update model embeddings

# Encode input text
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors='pt', padding=True) # Add padding

# Create an attention mask (1 for actual tokens, 0 for padding tokens)
attention_mask = input_ids.ne(tokenizer.pad_token_id).long()

# Generate text
output = model.generate(input_ids, attention_mask=attention_mask, max_length=100, num_return_sequences=1)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

##trainer script

In [ ]:
!pip install datasets transformers
! pip install -U accelerate
! pip install -U transformers

if __name__ == "__main__":
    from datasets import load_dataset
    from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments, GPT2LMHeadModel

    # Load a dataset
    dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')

    # Initialize the tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    tokenizer.pad_token = tokenizer.eos_token

    # Function to tokenize the dataset
    def tokenize_function(examples, tokenizer=tokenizer):
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

    # Tokenize the dataset
    tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

    # Initialize the Data Collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    # Initialize the model
    model = GPT2LMHeadModel.from_pretrained('gpt2')

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=2,
        save_steps=10_000,
        save_total_limit=2,
        prediction_loss_only=True,
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset,
    )

    # Train the model
    trainer.train()


